In [529]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, chi2, truncnorm, multivariate_normal
from tqdm import trange
from sklearn import linear_model
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from densratio import densratio
from numpy import linalg as la
import matplotlib.pyplot as plt

# Functions

In [565]:
import numpy as np
from sklearn.linear_model import LogisticRegression

def density_ratio_estimate_prob(D_nu, D_de):
    l_nu = np.ones(len(D_nu))
    l_de = np.zeros(len(D_de))
    
    l = np.concatenate((l_nu, l_de))
    D = np.concatenate((D_nu, D_de))
    
    #fit losgistic model
    C = 0.1
    model = LogisticRegression(penalty= 'l2', C= C)
    model.fit(D, l)
    
    # get density ratios for all samples
    density_ratios = (model.predict_proba(D_de)[:, 1] / model.predict_proba(D_nu)[:, 0])*(len(D_de)/len(D_nu))
    
    return density_ratios
    


# def Covariate_Shift_Weight(x, z, v = 0):
#     return np.exp(((x - z @ s)**2 - (x - z @ t)**2)/2)

def Model_X(z, v, u):
    return z @ u + np.random.normal(0, 1, 1)

def T_statistic(y, x, z, v, u,regr):
#     d_y = regr.predict(z.reshape(1,20))
#     d_x = z@u
#     # coef = np.abs(LinearRegression(fit_intercept= False).fit(x.reshape(-1,1), y-d_y).coef_)
#     # print([coef, y-d_y, x])
    
#     return np.abs((y-d_y)*(x-d_x))
    return (y - z.sum()- x)**2
    # return coef[0]

def Conterfeits(y, x, z, v, u, L, K, regr):
    M = L * K - 1
    cnt = 0
    t_stat = T_statistic(y, x, z, v, u,regr)
    
    for i in range(M):
        x_ = Model_X(z, v, u)
        if t_stat > T_statistic(y, x_, z, v, u, regr):
            cnt=cnt+1
            
    return cnt // K

def PCRtest(Y, X, Z, V, u, s, t, L, K, covariate_shift, density_ratio, regr):
    n = Y.size
    W = np.array([0.0]*L)

    for j in range(n):
        y, x, z, v = Y[j], X[j], Z[j], V[j]
        if covariate_shift == True:
            ind = Conterfeits(y, x, z, v, u, L, K, regr)
            # W[ind] += density_ratio[j]
            W[ind] += true_density_ratio(x, z, v, s, t)
        if covariate_shift == False:
            W[Conterfeits(y, x, z, v, u, L, K, regr)] += 1
    return W, L/n * np.dot(W - n/L, W - n/L)

def generate_cov_matrix(Y, X, Z, V,u, s, t, L, K, density_ratio, regr):
    """
    Generate a covariance matrix for quadratic form normal rv.

    Parameters:
    - L (int): The size of the covariance matrix.

    Returns:
    - covariance_matrix (ndarray): The generated covariance matrix.
    """
    n = Y.size
    diag = np.array([0.0]*L)
    
    for j in range(n):
        y, x, z, v = Y[j], X[j], Z[j], V[j]
        # diag[Conterfeits(y, x, z, v, L, K, regr)] += (density_ratio[j]**2)
        diag[Conterfeits(y, x, z, v,u, L, K, regr)] += (true_density_ratio(x,z,v,s,t)**2)
    diag = L*(diag/n)- 1/L
    covariance_matrix = np.full((L, L), -1/L)  # Fill all entries with 1/L
    np.fill_diagonal(covariance_matrix, diag)  # Set diagonal entries to 1 - 1/L^2
    return covariance_matrix
    

In [566]:
import scipy.stats as stats

def chi_squared_p_value(chi_squared_statistic, df):
    """
    Calculate the p-value from a chi-squared distribution.

    Parameters:
    - chi_squared_statistic (float): The observed chi-squared test statistic.
    - df (int): The degrees of freedom.

    Returns:
    - p_value (float): The calculated p-value.
    """
    p_value = 1 - stats.chi2.cdf(chi_squared_statistic, df)
    return p_value

def monte_carlo_p_value(n_samples, covariance_matrix, L, quantile):
    """
    Calculate the probability corresponding to a given quantile using the Monte Carlo method.

    Parameters:
    - n_samples (int): The number of Monte Carlo samples to generate.
    - covariance_matrix (ndarray): The covariance matrix of the random vector X.
    - L (int): The number of components to sum.
    - quantile (float): The quantile value.

    Returns:
    - probability (float): The estimated probability corresponding to the quantile.
    """
    count = 0
    for _ in range(n_samples):
        sample = np.random.multivariate_normal(np.zeros(L), covariance_matrix)
        squared_sum = np.sum(sample**2)
        if squared_sum <= quantile:
            count += 1

    probability = count / n_samples
    return 1-probability


# 生成数据

In [602]:
# Generate Data

def generate(n, p, s, t, u, Alpha = 0):
    Z_source, Z_target = np.zeros((n, p)), np.zeros((n, p))
    V_source, V_target = 0, 0
    for i in range(n):
        Z_source[i] = np.random.normal(0, 1, p)
        Z_target[i] = np.random.normal(0.1, 1, p)
        
    X_source = Z_source @ u + np.random.normal(0, 1, n)
    X_target = Z_target @ u + np.random.normal(0, 1, n)

    V_source = Z_source @ s + X_source + np.random.normal(0, 10, n)
    V_target = Z_target @ t - X_target + np.random.normal(0, 10, n)
    
    Y_source, Y_target = np.zeros(n), np.zeros(n)
    for i in range(n):
        Y_source[i] = Z_source[i].sum() + X_source[i] + V_source[i] + np.random.normal(0, 1, 1) + Alpha * X_source[i]
        Y_target[i] = Z_target[i].sum() + X_target[i] + V_target[i] + np.random.normal(0, 1, 1) + Alpha * X_target[i]
    return Y_source.reshape(-1,1), X_source.reshape(-1,1), V_source.reshape(-1,1), Z_source,\
           Y_target.reshape(-1,1), X_target.reshape(-1,1), V_target.reshape(-1,1), Z_target


def true_density_ratio(X, Z, V, s, t):
    zs_prob = multivariate_normal.pdf(Z, mean=np.zeros(20), cov= np.identity(20))
    vs_prob = norm.pdf(V, loc=Z@s + X, scale=10)
    zt_prob = multivariate_normal.pdf(Z, mean= 0.1* np.ones(20), cov=np.identity(20))
    vt_prob = norm.pdf(V, loc=Z@t - X, scale=10)
    # print(zs_prob)
    # print(vs_prob)
    # print(zt_prob)
    # print(vt_prob)
    return (zt_prob*vt_prob)/(zs_prob*vs_prob)
    # return np.exp(0.5*Z.sum() - 0.25*(1/2) - (1/200)*(V- t@Z +X)**2 + (1/200)*(V - s@Z - X)**2)

# Test procedure

In [603]:
K = 30
L = 10
n, p = 2000, 20
s = np.random.normal(0, 1, p)
t = np.random.normal(0, 1, p)
u = np.random.normal(0, 1, p)

In [604]:
#generate data
Y_source, X_source, V_source, Z_source, Y_target, X_target, V_target, Z_target = generate(n, p, s, t, u, 0)

# calculate density ratio
D_s = np.concatenate((X_source, Z_source, V_source), axis = 1)
D_t = np.concatenate((X_target, Z_target, V_target), axis = 1)
densratio_obj = densratio(D_t, D_s)

#calculate density ratio for each sample
sample_density_ratio = densratio_obj.compute_density_ratio(D_s)




RuLSIF starting...
Searching for the optimal sigma and lambda...
sigma = 0.00100, lambda = 0.00100, score = 0.00000
sigma = 0.00100, lambda = 0.01000, score = -0.00000
sigma = 0.00100, lambda = 0.10000, score = 0.00000
sigma = 0.00100, lambda = 1.00000, score = 0.00000
sigma = 0.00100, lambda = 10.00000, score = 0.00000
sigma = 0.00100, lambda = 100.00000, score = 0.00000
sigma = 0.00100, lambda = 1000.00000, score = 0.00000
sigma = 0.00100, lambda = 10000.00000, score = -0.00000
sigma = 0.00100, lambda = 100000.00000, score = 0.00000
sigma = 0.00100, lambda = 1000000.00000, score = 0.00000
sigma = 0.00100, lambda = 10000000.00000, score = -0.00000
sigma = 0.00100, lambda = 100000000.00000, score = 0.00000
sigma = 0.00100, lambda = 1000000000.00000, score = 0.00000
sigma = 0.01000, lambda = 0.00100, score = 0.00000
sigma = 0.01000, lambda = 0.01000, score = -0.00000
sigma = 0.01000, lambda = 0.10000, score = 0.00000
sigma = 0.01000, lambda = 1.00000, score = 0.00000
sigma = 0.01000, la

In [594]:
# distill information of Z on Y
reg = LassoCV().fit(Z_source,Y_source)

reg.coef_

# #make random forest for statistic
# from sklearn.ensemble import RandomForestRegressor

# Ireg = RandomForestRegressor(n_estimators=20)
# Ireg.fit(Z_source, Y_source)

#Fit neural network for statistics


/Users/bowenxu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([-0.39010129, -1.55838832,  4.93204043, -0.43669011, -1.55059672,
       -0.19307776,  4.11488835, -2.09585992, -0.65456078,  3.59275488,
        1.63229574,  2.57788204,  2.93425853,  1.64019078, -0.6599822 ,
        0.17845489, -1.01788505,  0.2576904 ,  1.65782141,  1.68082479])

In [587]:
reg.score(Z_source, Y_source)

0.9363575561132237

In [605]:
sum = 0
for i in range(1000):
    sum += true_density_ratio(X_source[i], Z_source[i], V_source[i], s, t)
    # print(true_density_ratio(X_source[i], Z_source[i], V_source[i], s, t))
    # print(Z_source[i]@s + X_source[i] - Z_source[i]@t + X_source[i])
sum/1000

array([1.03789167])

In [392]:
print(max(sample_density_ratio))
print(min(sample_density_ratio))
mean = np.mean(sample_density_ratio)
print(mean)
# sample_density_ratio = sample_density_ratio/mean

1.2570901125932832
0.011365891675698814
0.8760320576697079


In [137]:
PCRtest(Y_target, X_target, Z_target,V_target, L = 10, K = 30, covariate_shift = False, density_ratio = sample_density_ratio, regr = Ireg)

TypeError: PCRtest() got an unexpected keyword argument 'reg'

In [606]:
PCRtest(Y_source, X_source, Z_source,V_source, u, s, t,L = 10, K = 20, covariate_shift = False, density_ratio = sample_density_ratio, regr = reg)

(array([250., 221., 245., 222., 185., 196., 185., 179., 159., 158.]), 49.01)

In [556]:
chi_squared_p_value(14, 9)

0.12232522803866253

In [590]:
# verifrandomion by the p value
l = 10
count = 0
for j in trange(500):
    
    cov1 = generate_cov_matrix(Y_source, X_source, Z_source,V_source,u,s,t, L = l, K =30, density_ratio = sample_density_ratio, regr = reg)
    w, statistic = PCRtest(Y_source, X_source, Z_source,V_source,u,s,t, L = l, K = 30, covariate_shift = True, density_ratio = sample_density_ratio, regr = reg)
    print(statistic)
    p_value = monte_carlo_p_value(100000, cov1, l, statistic)
    print(p_value)
    if p_value < 0.1:
        count += 1
probability = count/500


  0%|          | 0/500 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
PCRtest(Y_source, X_source, Z_source, V_source, L = 10, K = 20, covariate_shift = True)

In [ ]:
*write the function

In [ ]:
def multiple_test():

In [ ]:
# previous code

# def int_(x):
#     if x >= 80: return 79
#     return int(x)

# def Many_Tests(m, Alpha):
#     X = []
#     n, p=500, 20
#     for i in trange(m):
#         s = np.random.normal(0, 1, p)
#         t = s + np.random.normal(0, 0.1, p)
#         u = np.random.normal(0, 1, p)
#         Y_source, X_source, V_source, Z_source, Y_target, X_target, V_target, Z_target = generate(n, p, s, t, u, Alpha)
#         u, v = PCRtest(Y_source, X_source, Z_source, L = 5, K = 20, covariate_shift = False)
#         X.append(v)
#     return X
# def Density_Variance(n):
#     a, b=0, 0
#     for i in range(n):
#         z = np.random.normal(0, 1, p)
#         x = z @ s + np.random.normal(0, 1, 1)
#         a += Covariate_Shift_Weight(x, z)**2
#         b += np.exp((z @ t - z @ s)**2)
#     return a/n, b/n

# Power simulation

1.First, we set the rejection threshold to be 0.05. We will run 1000 simulations to estimate the power and average the results over 1000 trials. Plot the power w.r.t L with fixed K = 50.

In [ ]:
# simulation code
K = 20
L = 10
n, p = 1000, 20
s = np.random.normal(0, 1, p)
t = s + np.random.normal(0, 0.1, p)
u = np.random.normal(0, 1, p)

result = []
for l in range(4, 30):
    for i in range
    cov1 = generate_cov_matrix(Y_source, X_source, Z_source, L = l, K = 20)
    result.append(monte_carlo_p_value(100000, cov1, l, PCRtest(Y_source, X_source, Z_source, L = l, K = 20, covariate_shift = True)[1]))
    
    

In [303]:
x = np.random.normal(0,25,500)
y = np.random.normal(2,1,500)
sum = 0
for i in range(500):
    sum += norm.pdf(x[i], loc = 5, scale = 25)/norm.pdf(x[i], loc = 0, scale = 25)
sum/500

1.0139707729582619